# #4 Darts - Formulation

_Author: Andrea Rujano_  
_August, 2021_

This is the MIP formulation of the puzzle. Statement and solution implementation of all puzzles 
are available from the main page of the [Fun Puzzles](https://mip-master.github.io/puzzles/) project, 
which is maintained by [Mip Master](https://mipmaster.org/).

Let's start by reviewing the statement of the puzzle:  
There are three people darts: Andrea, Antonio, and Luiz. They threw 6 darts each (red marks in the figure), 
and each scored 71 points. We also know that Andrea’s first 2 darts scored 22 points and Antonio’s first dart scored 3 points. We need to find out which player hit the bullseye.


![Darts](../../figures/4_darts.png)  

## Input Data

We start by defining three sets of indices.  
- The first set corresponds to the **player** (Andrea, Antonio, and Luiz, respectively):  
    `I = {1, 2, 3}` 
- Next, we define the set of **shots** (each player threw 6 darts):  
    `J = {1, 2, 3, 4, 5, 6}`
- Next, we define the set of **scores** (scoring regions of the dartboard):  
    `K = {1, 2, 3, 5, 10, 20, 25, 50}`
- Finally, we define a dictionary which indicates the numbers of **marks** in each **scoring region** (as shown in the figure):  
`R = {1: 3, 2: 2, 3: 2, 5: 2, 10: 3, 20: 3, 25: 2, 50: 1}`

## Decision variables

With the sets of indices defined above we can now define the decision variables. Can you think how we do this?

We want to answer is this: What is the score of each player in each shot?

So we define decision variables with three indices, and they are all binary variables:
- $x_{ijk}$ equals $1$ if player $i$ scores $k$ in shot $j$, $0$ otherwise.

For example, if the variable $x_{1,2,5}$ equals one, it means that player $1$ (Andrea in this case) scores $5$ in her second shot. We assume that in every shot the player hits the dartboard, so they always score at least one point.

## Constraints

With the definition of the decision variables above, we can now model the rules of the game with constraints.

- *Every shot hits one, and only one, scoring region*.
$$ \sum_{k} x_{ijk} = 1, \; \forall i, j.$$ 
For example, for $i=1$ and $j=2$, this constraint becomes
$$x_{1,2,1} + x_{1,2,2} + x_{1,2,3} + x_{1,2,5} + x_{1,2,10} + x_{1,2,20} + x_{1,2,50} = 1,$$
which means that, out of the seven possible scores, the first player in her second shot can only hit one score. In fact, the only way for a sum of binary variables to add up to one is that one of those variables equal one and all the other variables equal zero.

- *Every player scored $71$ points*.
$$ \sum_{j, k}k\cdot x_{ijk}=71, \; \forall i.$$
This constraint is saying that the total score of player $i$, across all shots and all possible scores in each shot, must add up to $71$. Notice that we multiply $x_{ijk}$ by $k$ so that, if $x_{ijk} = 1$, we add $k$ to the sum, and if $x_{ijk} = 0$, we add zero.

- *Number of marks in each scoring region*.  
$$ \sum_{i, j} x_{ijk} = R[k], \; \forall k.$$
This is saying that for $k=25$, for example, there are $R[25]=2$ darts, across all players and all shots, that hit the scoring region of $25$ points. This constraint, in combination with the next two constraints, makes the solution to this puzzle to be unique.

- *Andrea's first two shots scored 22 points*. 
$$ \sum_{k} x_{11k} + x_{12k} = 22.$$
This constraint is identical to the constraint "*Every player scored $71$ points*", except that this one is restricted to player $1$ and its first two shots.  
Alternatively, given that $22$ points can only be achieved in two shots if one dart hits $20$ and the other dart hits $2$, we could simply enforce $x_{1,1,20}=1$ and $x_{1,2,2}=1$ (or $x_{1,1,2}=1$ and $x_{1,2,20}=1$).

- *Antonio's first shot scored 3 points*.
$$ x_{2,1,3}=1.$$

## Objective Function

This is just a feasibility problem, since there is nothing to optimize. Hence, we can simply define a dummy objective function such as the sum of all variables. The sum of all variables may sounds like a sophisticated choice for a dummy objective function, but this can help the solver when the problem is hard to solve (although this is not the case with this baby problem).

$$ \min \sum_{i,j,k}x_{ijk}.$$

## Final formulation

$$ 
\begin{array}{rrl}
\min & \sum_{i,j,k}x_{ijk} &  \\
\mbox{s.t.} & \sum_{k} x_{ijk} = 1, & \forall i, j\\
            & \sum_{j, k}k\cdot x_{ijk}=71, & \forall i\\
            & \sum_{i, j} x_{ijk} = R[k], & \forall k\\
            & \sum_{k} x_{11k} + x_{12k} = 22 & \\
            & x_{2,1,3}=1  & \\
            & x_{ijk} \in \{0,1\} & \forall i, j, k.\\
\end{array}
$$